In [ ]:
from chatbot import Chatbot

In [ ]:
# hyperparameters
base_model = 'NousResearch/Meta-Llama-3-8B'
prompt_format = 'llama'
device_map = 'cuda:0'
trained_model = ''

In [ ]:
# load the chatbot
chatbot = Chatbot(base_model, trained_model, device=device_map, format=prompt_format)

In [ ]:
# Run text generation pipeline with model
prompt = "Tell me about yourself."
result = chatbot.generate(prompt)
print("Prompt: ", prompt)
print("Response: ", result)